In [1]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce
from scipy.spatial import distance
import pathlib
import os
from sklearn.decomposition import PCA
import json
import random
from random import sample

random.seed(30)

In [2]:
# Set data input folder
input_folder = "inputs"

# Set output folder, subfolder
output_folder = "outputs"
if not os.path.exists(output_folder):
    os.makedirs(output_folder, exist_ok=True)

In [3]:
# Import the guide-level profiles
df_guide = pd.read_csv("outputs/20240202_6W_CP498_SABER_Pilot_HeLa_guide_normalized_merged_feature_select_median_ALLWELLS_cp.csv.gz")
# Subset the nontargeting guide profiles 
df_nontargeting = df_guide.query("Metadata_Foci_Barcode_MatchedTo_GeneCode == 'nontargeting'")

# Load hits from the hit calling process
whole_cell_hits = pd.read_csv('outputs/HeLa_CP_plate_level_median_per_feat_sig_genes_5_FDR_whole_cell_hits.csv')
comp_spec_hits = pd.read_csv('outputs/HeLa_CP_plate_level_median_per_feat_sig_genes_5_FDR_compartment_specific_hits.csv')
all_hits = pd.concat([whole_cell_hits,comp_spec_hits])
hit_list = list(comp_spec_hits.Gene) + list(whole_cell_hits.Gene)
whole_cell_hit_list = list(whole_cell_hits.Gene)
comp_spec_hit_list = list(comp_spec_hits.Gene)

# list non hit genes
all_genes_list = list(df_guide.Metadata_Foci_Barcode_MatchedTo_GeneCode.unique())
all_genes_list.remove("nontargeting")
non_hit_list = [gene for gene in all_genes_list if gene not in hit_list]
print("All genes:",len(all_genes_list),"non-hit genes",len(non_hit_list),'Whole cell hits',len(whole_cell_hit_list),'Compartment hits',len(comp_spec_hit_list))


All genes: 590 non-hit genes 189 Whole cell hits 379 Compartment hits 22


In [4]:
df_temp = df_guide.copy(deep=True)
features = list(df_guide.columns)
gene_list = list(df_temp.Metadata_Foci_Barcode_MatchedTo_GeneCode)
df_temp = df_temp.drop('Metadata_Foci_Barcode_MatchedTo_GeneCode',axis=1).set_index('Metadata_Foci_Barcode_MatchedTo_Barcode')
df_temp = df_temp.reset_index()
df_temp["Metadata_Foci_Barcode_MatchedTo_GeneCode"] = gene_list
df_temp = df_temp[features]
df_temp

,Metadata_Foci_Barcode_MatchedTo_GeneCode,Metadata_Foci_Barcode_MatchedTo_Barcode,Cells_AreaShape_BoundingBoxMinimum_X,Cells_AreaShape_BoundingBoxMinimum_Y,Cells_AreaShape_CentralMoment_0_1,Cells_AreaShape_CentralMoment_0_3,Cells_AreaShape_CentralMoment_1_0,Cells_AreaShape_CentralMoment_1_2,Cells_AreaShape_CentralMoment_1_3,Cells_AreaShape_CentralMoment_2_1,...,Nuclei_Texture_InfoMeas1_Mito_5_01_256,Nuclei_Texture_InfoMeas1_Mito_5_03_256,Nuclei_Texture_InfoMeas1_WGA_10_00_256,Nuclei_Texture_InfoMeas1_WGA_5_00_256,Nuclei_Texture_InfoMeas1_WGA_5_02_256,Nuclei_Texture_InfoMeas2_DAPI_Painting_10_03_256,Nuclei_Texture_InfoMeas2_Mito_5_02_256,Nuclei_Texture_InfoMeas2_Phalloidin_5_00_256,Nuclei_Texture_SumAverage_DAPI_Painting_10_03_256,Nuclei_Texture_SumVariance_Mito_10_03_256
0,AARS2,AAAGGCGGCCCTCACGGCCG,-0.049695,-0.007310,0.068068,0.282360,-0.716255,0.005817,0.218232,-0.154402,...,0.327775,0.296771,0.487940,0.305170,0.191921,0.154430,-0.252812,-0.493225,0.296880,-0.279502
1,AARS2,AGCAAACTGGGGTCGCCGCG,-0.056370,0.034850,-0.129720,0.102712,-0.227415,0.196801,0.005880,-0.064720,...,-0.125585,-0.222343,-0.267363,-0.014090,-0.084492,0.081421,0.078580,0.115098,-0.156265,-0.170241
2,AARS2,CCAACTTCTACGCAGAACAG,-0.160030,-0.091385,0.148568,-0.216035,0.034805,0.163045,0.066160,0.356585,...,-0.402410,-0.458070,-0.447815,-0.317605,-0.398200,0.132896,0.480540,0.317265,-0.341360,0.114366
3,AARS2,GCTGAGCCAGTTCAGAAGCA,0.286901,0.142030,-0.313969,-0.053498,0.153567,-0.259566,0.012046,-0.067856,...,0.165980,-0.044400,-0.000590,0.041017,0.098723,0.022136,-0.142982,-0.108239,-0.352795,-0.215195
4,AARSD1,ACCTCCGCTCCCAATCTACC,-0.135745,-0.062917,-0.331497,0.277325,0.218805,0.064993,-0.232255,-0.474220,...,0.639800,0.435405,0.689870,0.815490,0.522225,-0.641835,-0.834870,-0.789625,-0.769190,-0.571055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,nontargeting,TAAGATCCGCGGGTGGCAAC,-0.471880,0.069689,0.247578,-0.353750,-0.685932,-0.330715,-0.401865,0.184790,...,0.359865,0.634615,0.465790,0.498780,0.270355,0.246790,-0.308400,-0.426650,0.200284,-0.234015
2396,nontargeting,TCCCGGTTGGTGAACGATAC,0.023480,-0.463288,0.601885,0.030910,0.374675,-0.112565,0.301557,-0.650605,...,0.656175,0.597540,0.525310,0.580275,0.617890,0.059865,-0.553425,-0.592625,-0.313280,-0.296670
2397,nontargeting,TGCCGTGAAAAGACGCTGCG,-0.513425,-0.240830,0.340108,0.132513,-0.012648,0.194555,0.042534,-0.164266,...,0.235540,0.400525,0.278735,0.500875,0.429855,-0.129978,-0.430115,-0.451555,-0.036847,-0.352740
2398,nontargeting,TGGCCACGAATTCCGCCGCC,-0.011335,-0.114445,0.042562,-0.008606,0.443205,0.138462,0.008197,0.031328,...,0.500430,0.425680,0.443780,0.492115,0.476090,0.398430,-0.498765,-0.494330,0.441350,-0.380610


In [5]:
df_temp = df_guide.drop('Metadata_Foci_Barcode_MatchedTo_GeneCode',axis=1).set_index('Metadata_Foci_Barcode_MatchedTo_Barcode')
# Perform principal component analysis on hit list
pca = PCA()
pca.fit(df_temp)
x = list(pca.explained_variance_ratio_)
# Find principal component that represents 90% variation
PCA_lookup = {}
for i in range(len(x)):
    distance = abs(.9-sum(x[:i+1]))
    PCA_lookup[distance] = i 
component = PCA_lookup[min(PCA_lookup.keys())]+1
print (f'Principal component representing closest to 90% variation is {component}')
# Perform principal component analysis and select components representing 90% of variation in data
pca = PCA(n_components=component)
df_guide_pca = pd.DataFrame(pca.fit_transform(df_temp),index=df_temp.index)
df_guide_pca.head()

Principal component representing closest to 90% variation is 102


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,101
Metadata_Foci_Barcode_MatchedTo_Barcode,,,,,,,,,,,,,,,,,,,,,
AAAGGCGGCCCTCACGGCCG,0.490518,3.686846,-2.171395,-1.993583,-0.866748,-1.599464,0.480718,-0.035576,0.265111,0.184354,...,-0.292537,0.446860,-0.271221,0.104961,-0.637043,0.135344,-0.170325,-0.589620,-0.269927,-0.436197
AGCAAACTGGGGTCGCCGCG,-0.595151,0.286398,-1.600191,-0.321986,-0.368362,1.424164,1.720628,1.428887,-1.090676,-0.981123,...,-0.161702,-0.079624,-0.330768,-0.271000,0.134448,0.124433,0.133492,0.063175,-0.277479,0.169217
CCAACTTCTACGCAGAACAG,0.963539,2.582813,-0.102827,0.520521,-0.945019,0.413353,2.533179,1.987970,-1.508221,-1.348158,...,0.054551,-0.282844,-0.245120,-0.209199,0.266384,-0.114417,0.152727,0.098827,-0.085751,0.188986
GCTGAGCCAGTTCAGAAGCA,0.259027,2.793519,-3.492034,1.862051,-1.909145,-0.269427,2.377853,1.795669,-0.867349,-0.130634,...,-0.479328,0.107960,-0.440232,-0.151283,-0.044378,-0.351262,0.016248,-0.097878,-0.100056,0.190636
ACCTCCGCTCCCAATCTACC,-8.208768,-5.449111,-2.653009,-1.016760,-2.355556,3.232852,-1.967302,-0.415276,-0.682730,2.383126,...,-0.438984,0.135878,-0.127172,0.065672,-0.668582,-0.231231,-0.611484,-0.217877,0.077861,0.048823


In [6]:
df_guide_pca_updated = df_guide_pca.reset_index()
pca_feat_list = list(df_guide_pca_updated.columns)
feat_list = ['Metadata_Foci_Barcode_MatchedTo_GeneCode']
feat_list.extend(pca_feat_list)
df_guide_pca_updated["Metadata_Foci_Barcode_MatchedTo_GeneCode"] = gene_list
df_guide_pca_updated = df_guide_pca_updated[feat_list]
df_guide_pca_updated

,Metadata_Foci_Barcode_MatchedTo_GeneCode,Metadata_Foci_Barcode_MatchedTo_Barcode,0,1,2,3,4,5,6,7,...,92,93,94,95,96,97,98,99,100,101
0,AARS2,AAAGGCGGCCCTCACGGCCG,0.490518,3.686846,-2.171395,-1.993583,-0.866748,-1.599464,0.480718,-0.035576,...,-0.292537,0.446860,-0.271221,0.104961,-0.637043,0.135344,-0.170325,-0.589620,-0.269927,-0.436197
1,AARS2,AGCAAACTGGGGTCGCCGCG,-0.595151,0.286398,-1.600191,-0.321986,-0.368362,1.424164,1.720628,1.428887,...,-0.161702,-0.079624,-0.330768,-0.271000,0.134448,0.124433,0.133492,0.063175,-0.277479,0.169217
2,AARS2,CCAACTTCTACGCAGAACAG,0.963539,2.582813,-0.102827,0.520521,-0.945019,0.413353,2.533179,1.987970,...,0.054551,-0.282844,-0.245120,-0.209199,0.266384,-0.114417,0.152727,0.098827,-0.085751,0.188986
3,AARS2,GCTGAGCCAGTTCAGAAGCA,0.259027,2.793519,-3.492034,1.862051,-1.909145,-0.269427,2.377853,1.795669,...,-0.479328,0.107960,-0.440232,-0.151283,-0.044378,-0.351262,0.016248,-0.097878,-0.100056,0.190636
4,AARSD1,ACCTCCGCTCCCAATCTACC,-8.208768,-5.449111,-2.653009,-1.016760,-2.355556,3.232852,-1.967302,-0.415276,...,-0.438984,0.135878,-0.127172,0.065672,-0.668582,-0.231231,-0.611484,-0.217877,0.077861,0.048823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,nontargeting,TAAGATCCGCGGGTGGCAAC,-3.437158,-1.917380,-1.509893,-6.304269,1.405276,1.323337,-2.085760,0.410990,...,0.076760,-0.547628,-0.192380,0.631735,-0.098925,0.592741,0.137949,0.427153,0.202603,0.009115
2396,nontargeting,TCCCGGTTGGTGAACGATAC,-3.430271,-2.732776,-4.302808,-6.083132,-1.559545,0.767722,2.057993,2.095849,...,-0.358055,0.739089,-0.236610,0.062489,0.188860,-0.482972,0.127081,0.185295,-0.107594,-0.534843
2397,nontargeting,TGCCGTGAAAAGACGCTGCG,-4.795058,1.198451,-2.267250,-1.555450,-0.321734,-1.636284,0.330087,0.992569,...,-0.344208,0.278669,0.083133,-0.331621,0.001723,-0.133797,0.029453,0.084123,0.176028,0.297220
2398,nontargeting,TGGCCACGAATTCCGCCGCC,0.771068,6.087222,-3.422265,-2.934733,0.422450,-2.446637,0.737432,0.045173,...,0.027615,0.227532,0.145162,-0.057901,0.398882,0.098418,-0.056778,-0.217555,0.277777,-0.138877


In [7]:
def calculate_mean_similarity(cosine_array):
    total_sum = 0
    for i in range(4):
        similarities = cosine_array[i]
        target_sum = 0
        for j in range(4):
            target_sum += similarities[j]  
        target_mean = ((target_sum-1)/3)
        total_sum += target_mean

    total_mean = total_sum / 4
    return total_mean

hit_guide_sim_list =[]
for i in range(len(whole_cell_hit_list)):
    df_temp = df_guide_pca_updated.query("Metadata_Foci_Barcode_MatchedTo_GeneCode == @whole_cell_hit_list[@i]")
    df_temp = df_temp.drop(['Metadata_Foci_Barcode_MatchedTo_Barcode'],axis=1)
    df_temp = df_temp.set_index("Metadata_Foci_Barcode_MatchedTo_GeneCode")
    cosine_array = cosine_similarity(df_temp)
    hit_guide_sim = calculate_mean_similarity(cosine_array)
    hit_guide_sim_list.append(hit_guide_sim)
average_cosine_distance = sum(hit_guide_sim_list)/len(hit_guide_sim_list)
print('Average cosine distance: ',average_cosine_distance)

Average cosine distance:  0.417211379580346


In [7]:
def cosine_to_df(df_temp, cosine_array, i):
    cosine_list = cosine_array[i]
    gene_list = list(df_temp.index)
    cosine_df = pd.DataFrame(index=gene_list)
    cosine_df['cosine'] = cosine_list
    cosine_df = cosine_df.sort_values('cosine',ascending=False)   
    return cosine_df

def ap_from_cosine_df(cosine_df,gene,n=10):    
    #print(cosine_df.iloc[:20])
    index_list = list(cosine_df.index)
    boolean = [1 if  i == gene else 0 for i in index_list ]
    grades_list=[]
    for i in range(2,n+2):
        pre_grade = sum(boolean[1:i])/(i-1)
        grades_list.append(pre_grade*boolean[i-1])
    return sum(grades_list)/3

def calculate_map(df_guide, gene):
    df_temp = df_guide.query("Metadata_Foci_Barcode_MatchedTo_GeneCode == 'nontargeting' | Metadata_Foci_Barcode_MatchedTo_GeneCode == @gene")
    df_temp = df_temp.drop(['Metadata_Foci_Barcode_MatchedTo_Barcode'],axis=1)
    df_temp = df_temp.set_index("Metadata_Foci_Barcode_MatchedTo_GeneCode")
    #print(df_temp)
    ap_list = []
    cosine_array = cosine_similarity(df_temp)
    for guide in range(4):
        cosine_df = cosine_to_df(df_temp, cosine_array, guide)
        #print(cosine_df[:10])
        guide_ap = ap_from_cosine_df(cosine_df,gene,10)
        ap_list.append(guide_ap)
    return np.mean(ap_list)

In [18]:
genes_list = all_genes_list
map_list = []
for i in range(len(genes_list)):
    gene = genes_list[i]
    #print(f"Calculating mean average precision for gene: {gene}")
    gene_map = calculate_map(df_guide_pca_updated, gene)
    #map_list.append([gene, gene_map])
    map_list.append(gene_map)
print(f'For all genes ({len(all_genes_list)} genes) the mAP values is',np.mean(map_list))

For all genes (590 genes) the mAP values is 0.407900636714196


In [19]:
genes_list = whole_cell_hit_list
map_list = []
for i in range(len(genes_list)):
    gene = genes_list[i]
    #print(f"Calculating mean average precision for gene: {gene}")
    gene_map = calculate_map(df_guide_pca_updated, gene)
    #map_list.append([gene, gene_map])
    map_list.append(gene_map)
print(f'For whole cell hits ({len(whole_cell_hit_list)} genes) the mAP values is',np.mean(map_list))

For whole cell hits (379 genes) the mAP values is 0.5579229837640128


In [20]:
genes_list = comp_spec_hit_list
map_list = []
for i in range(len(genes_list)):
    gene = genes_list[i]
    #print(f"Calculating mean average precision for gene: {gene}")
    gene_map = calculate_map(df_guide_pca_updated, gene)
    #map_list.append([gene, gene_map])
    map_list.append(gene_map)
print(f'For compartment hits ({len(comp_spec_hit_list)} genes) the mAP values is',np.mean(map_list))

For compartment hits (22 genes) the mAP values is 0.18962542087542086


In [21]:
genes_list = non_hit_list
map_list = []
for i in range(len(genes_list)):
    gene = genes_list[i]
    #print(f"Calculating mean average precision for gene: {gene}")
    gene_map = calculate_map(df_guide_pca_updated, gene)
    #map_list.append([gene, gene_map])
    map_list.append(gene_map)
print(f'For non hits ({len(non_hit_list)} genes) the mAP values is',np.mean(map_list))

For non hits (189 genes) the mAP values is 0.13246987066431506
